In [2]:
import pandas as pd
import numpy as np

from PIL import Image
import os
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input,Lambda,Flatten,Dense
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [3]:
data= pd.read_excel('Cases Meta data.xlsx')
df = data[["CaseID", "VIA"]]
df=df.sort_values(by="CaseID")

In [4]:
# Step 1: Create a list of file paths
image_dir = 'D:\coding\PRML\hobby\cervical cancer\Data'  # Replace with your image directory
file_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir)]
print(file_paths[184])

D:\coding\PRML\hobby\cervical cancer\Data\APV1.jpg


In [5]:
df2= pd.read_excel('Cases - Images.xlsx')
df2

,CaseNumber,File,Type
0,1,AFC0.jpg,Before application of acetic acid
1,1,AFC1.jpg,After application of acetic acid
2,2,AJL0.jpg,Before application of acetic acid
3,2,AJL1.jpg,After application of acetic acid
4,3,AGY0.jpg,Before application of acetic acid
...,...,...,...
415,184,AEU1.jpg,After application of acetic acid
416,185,ADF0.jpg,Before application of acetic acid
417,185,ADF1.jpg,After application of acetic acid
418,186,ABK0.jpg,Before application of acetic acid


In [6]:

# Assuming you have a DataFrame 'df' with 'CaseID' column

# Step 1: Create a list of valid image file paths
image_dir = 'D:/coding/PRML/hobby/cervical cancer/Data'  # Replace with your image directory
valid_image_paths = []

# Create a list of indices to drop
indices_to_drop = []

for i, case_id in enumerate(df['CaseID']):
    image_filename = f"{case_id}1.jpg"  # Assuming images are named as 'caseID1.jpg'
    image_path = os.path.join(image_dir, image_filename)
    
    if os.path.exists(image_path):
        valid_image_paths.append(image_path)
    else:
        indices_to_drop.append(i)

# Drop rows with missing images
df = df.drop(index=indices_to_drop)

# Add the "Path" column to the DataFrame
df["Path"] = valid_image_paths

# Display the updated DataFrame
print(df.head())

    CaseID                   VIA  \
86     AAA              Negative   
147    AAB              Positive   
180    AAC  Suspicious of cancer   
96     AAD              Positive   
123    AAE              Positive   

                                                  Path  
86   D:/coding/PRML/hobby/cervical cancer/Data\AAA1...  
147  D:/coding/PRML/hobby/cervical cancer/Data\AAB1...  
180  D:/coding/PRML/hobby/cervical cancer/Data\AAC1...  
96   D:/coding/PRML/hobby/cervical cancer/Data\AAD1...  
123  D:/coding/PRML/hobby/cervical cancer/Data\AAE1...  


In [27]:
df["Image"]=df["Path"].map(lambda x: np.asarray(Image.open(x)))
df

,CaseID,VIA,Path,Image
86,AAA,0,D:/coding/PRML/hobby/cervical cancer/Data\AAA1...,"[[[202, 77, 29], [199, 77, 28], [195, 73, 24],..."
147,AAB,1,D:/coding/PRML/hobby/cervical cancer/Data\AAB1...,"[[[111, 50, 22], [111, 49, 24], [111, 50, 31],..."
180,AAC,2,D:/coding/PRML/hobby/cervical cancer/Data\AAC1...,"[[[103, 80, 62], [112, 89, 71], [118, 93, 73],..."
96,AAD,1,D:/coding/PRML/hobby/cervical cancer/Data\AAD1...,"[[[100, 62, 15], [105, 67, 20], [119, 82, 37],..."
123,AAE,1,D:/coding/PRML/hobby/cervical cancer/Data\AAE1...,"[[[70, 22, 10], [48, 6, 0], [49, 19, 11], [62,..."
...,...,...,...,...
44,APF,0,D:/coding/PRML/hobby/cervical cancer/Data\APB1...,"[[[142, 112, 86], [143, 113, 87], [137, 110, 8..."
49,API,0,D:/coding/PRML/hobby/cervical cancer/Data\APF1...,"[[[255, 196, 163], [254, 194, 160], [251, 191,..."
55,APT,0,D:/coding/PRML/hobby/cervical cancer/Data\API1...,"[[[169, 92, 50], [171, 98, 53], [167, 103, 55]..."
56,APV,0,D:/coding/PRML/hobby/cervical cancer/Data\APT1...,"[[[126, 105, 58], [133, 109, 65], [132, 98, 61..."


In [28]:
df["Image"][1].shape

(72, 96, 3)

In [50]:
from keras.utils import to_categorical  # Added for one-hot encoding
df["VIA"] = to_categorical(df["VIA"],num_classes=3)

In [51]:
X_train,X_test,Y_train,Y_test= train_test_split(df[["Image"]],df["VIA"],test_size=0.2, random_state= 42)
X_train.head()
Y_train.head()

62     0.0
151    1.0
169    1.0
174    1.0
71     0.0
Name: VIA, dtype: float32

In [52]:
train_df=pd.concat([X_train,Y_train],axis=1)
train_df

,Image,VIA
62,"[[[83, 68, 49], [79, 60, 43], [69, 43, 28], [5...",0.0
151,"[[[86, 12, 47], [65, 0, 28], [70, 5, 39], [63,...",1.0
169,"[[[36, 13, 5], [32, 8, 0], [25, 0, 0], [30, 1,...",1.0
174,"[[[10, 4, 8], [23, 17, 21], [3, 0, 2], [3, 0, ...",1.0
71,"[[[106, 79, 52], [106, 79, 52], [111, 83, 61],...",0.0
...,...,...
67,"[[[120, 53, 1], [128, 62, 10], [122, 63, 7], [...",0.0
97,"[[[88, 0, 49], [94, 4, 55], [88, 0, 46], [95, ...",1.0
149,"[[[248, 175, 230], [166, 93, 146], [188, 111, ...",1.0
52,"[[[71, 39, 16], [71, 41, 17], [68, 42, 19], [7...",0.0


In [53]:
test_df=pd.concat([X_test,Y_test],axis=1)
test_df

,Image,VIA
144,"[[[59, 14, 34], [61, 14, 24], [68, 18, 11], [7...",1.0
122,"[[[132, 75, 108], [140, 94, 123], [112, 82, 11...",1.0
36,"[[[106, 85, 56], [111, 90, 61], [124, 98, 75],...",0.0
109,"[[[234, 140, 66], [238, 144, 70], [229, 135, 6...",1.0
14,"[[[168, 126, 150], [164, 120, 145], [163, 110,...",0.0
181,"[[[126, 93, 58], [129, 97, 59], [133, 97, 61],...",1.0
127,"[[[135, 70, 94], [112, 48, 72], [115, 51, 75],...",1.0
110,"[[[221, 255, 246], [124, 178, 144], [16, 73, 3...",1.0
183,"[[[74, 31, 58], [70, 27, 54], [65, 22, 49], [6...",1.0
77,"[[[28, 17, 13], [33, 18, 13], [27, 4, 0], [22,...",0.0


In [33]:
inputs=(720,960,3)
vgg=VGG16(include_top=False,input_shape= inputs,weights="imagenet")
vgg.output

<KerasTensor: shape=(None, 22, 30, 512) dtype=float32 (created by layer 'block5_pool')>

In [34]:
for layer in vgg.layers:
    layer.trainable=False

In [42]:
x=Flatten()(vgg.output)
prediction=Dense(3,activation="softmax")(x)
model=Model(inputs=vgg.inputs,outputs=prediction)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 720, 960, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 720, 960, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 720, 960, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 360, 480, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 360, 480, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 360, 480, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 180, 240, 128)     0   

 block5_conv3 (Conv2D)       (None, 45, 60, 512)       2359808   
                                                                 
 block5_pool (MaxPooling2D)  (None, 22, 30, 512)       0         
                                                                 
 flatten_2 (Flatten)         (None, 337920)            0         
                                                                 
 dense_2 (Dense)             (None, 3)                 1013763   
                                                                 
Total params: 15728451 (60.00 MB)
Trainable params: 1013763 (3.87 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [43]:
from keras import optimizers

adam=optimizers.Adam()

model.compile(loss="categorical_crossentropy",optimizer=adam,metrics=["accuracy"])

In [44]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

#data augmentation
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1.0/255.0,
    rotation_range=20,     # Random rotation within the range
    width_shift_range=0.1, # Random horizontal shift
    height_shift_range=0.1,  # Random vertical shift
    shear_range=0.2,      # Random shear
    zoom_range=0.2,       # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'   # Filling strategy for image transformations
)

# Assuming you have a DataFrame named train_df with columns 'image_array' and 'class_label'
train_data = np.stack(train_df['Image'].values)  # Convert image arrays to a numpy array
train_labels = train_df['VIA'].values

# Create a custom generator with data augmentation
def custom_generator(data, labels, datagen, batch_size):
    num_samples = len(data)
    while True:
        indices = np.random.choice(num_samples, batch_size)
        batch_data = data[indices]
        batch_labels = labels[indices]

        # Apply data augmentation to the batch of images
        augmented_batch = datagen.flow(batch_data, batch_labels, batch_size=batch_size)
        yield next(augmented_batch)

# Define batch size
batch_size = 32

# Create the generator
train_generator = custom_generator(train_data, train_labels, datagen, batch_size)

# You can use train_generator with model.fit


In [39]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Create an ImageDataGenerator with data augmentation
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1.0/255.0,
    rotation_range=20,     # Random rotation within the range
    width_shift_range=0.1, # Random horizontal shift
    height_shift_range=0.1,  # Random vertical shift
    shear_range=0.2,      # Random shear
    zoom_range=0.2,       # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'   # Filling strategy for image transformations
)

# Assuming you have a DataFrame named train_df with columns 'image_array' and 'class_label'
test_data = np.stack(test_df['Image'].values)  # Convert image arrays to a numpy array
test_labels = test_df['VIA'].values

# Create a custom generator with data augmentation
def custom_generator(data, labels, datagen, batch_size):
    num_samples = len(data)
    while True:
        indices = np.random.choice(num_samples, batch_size)
        batch_data = data[indices]
        batch_labels = labels[indices]

        # Apply data augmentation to the batch of images
        augmented_batch = datagen.flow(batch_data, batch_labels, batch_size=batch_size)
        yield next(augmented_batch)

# Define batch size
batch_size = 32

# Create the generator
test_generator = custom_generator(test_data, test_labels, datagen, batch_size)

# You can use train_generator with model.fit


In [40]:
from keras.callbacks import ModelCheckpoint
from datetime import datetime

checkpoint=ModelCheckpoint(filepath="model.keras",verbose=2,save_best_only=True)
callbacks=[checkpoint]

In [49]:
from datetime import datetime
from sklearn.metrics import accuracy_score
train_steps_per_epoch = len(train_data) // batch_size
validation_steps = len(test_data) // batch_size

# Now, you can use model.fit_generator
start = datetime.now()
model_history = model.fit(train_generator, steps_per_epoch=len(train_data)//batch_size, epochs=50, callbacks=callbacks, verbose=1)
duration = datetime.now() - start
print("Training completed in time:", duration)


Epoch 1/50


ValueError: in user code:

    File "c:\Users\sakth\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\sakth\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\sakth\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\sakth\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1127, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\sakth\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1185, in compute_loss
        return self.compiled_loss(
    F